# Definição de funções e imports necessários

In [1]:
import pandas as pd
import numpy as np
import json
import pymongo

### Funções

In [2]:
def readXLSX (path):
    return pd.read_excel(path, index_col=None);

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORD>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def deletingColumns (dataframe):
    columns = ["Região", 
               "UF", 
               "Código do Município", 
               "Nome do Município", 
               "Nome da Escola", 
               "Localização", 
               "Dependência Administrativa",
               "Identificador da Área"];
    
    for column in columns:
        try:
            dataframe.pop(column)
        except:
            continue
    
def JSONstructure (data):    
    dataJSON = {"Indicador de nível socioeconômico": 
                {
                "Última atualização": 2015,
                "Quantidade de Alunos com INSE": data ["Quantidade de Alunos com INSE calculado utilizado para o cálculo das médias"],
                "Média do Indicador de Nível Socioeconômico dos alunos": data["Média do Indicador de Nível Socioeconômico dos alunos"],
                "Classificação do Indicador de Nível Socioeconômico": data["Classificação do Indicador de Nível Socioeconômico"]
            }};

    return dataJSON;

def filterPelotas (dataframe):
    filter = dataframe["Código do Município"] == 4314407
    
    filterPelotas = dataframe[filter]
    filterPelotas.reset_index(drop=True, inplace=True)
    
    return filterPelotas

def insertIntoDB (dataframe, collection):
    for index, row in dataframe.iterrows():
        teste1 = JSONstructure(row);

        collection.update_one({"Código INEP": row["Código da Escola"]}, {"$set": teste1});

### Conexão com o BD:

In [3]:
collection = openDataBaseMongoDB()

### Extração dos dados

In [4]:
socioeconomicLevel = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\IndicadorNivelSocioeconomico.xlsx')

In [5]:
socioeconomicLevel.head()

,Código da Escola,Nome da Escola,UF,Código do Município,Identificador da Área,Dependência Administrativa,Localização,Quantidade de Alunos com INSE calculado utilizado para o cálculo das médias,Média do Indicador de Nível Socioeconômico dos alunos,Classificação do Indicador de Nível Socioeconômico
0,11000058,CENTRO DE ENSINO CLASSE A,Rondônia,1100205,1,4,1,130,64.17,Grupo 6
1,11000171,CENTRO EDUCACIONAL MOJUCA,Rondônia,1100205,1,4,1,17,54.96,Grupo 4
2,11000198,CENTRO EDUCACIONAL OBJETIVO,Rondônia,1100205,1,4,1,37,61.17,Grupo 5
3,11000201,EEEF PROF HERBERT DE ALENCAR,Rondônia,1100205,1,2,1,86,53.46,Grupo 4
4,11000244,COLEGIO DOM BOSCO,Rondônia,1100205,1,4,1,49,56.88,Grupo 5


In [6]:
socioeconomicLevel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63258 entries, 0 to 63257
Data columns (total 10 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   Código da Escola                                                             63258 non-null  int64  
 1   Nome da Escola                                                               63258 non-null  object 
 2   UF                                                                           63258 non-null  object 
 3   Código do Município                                                          63258 non-null  int64  
 4   Identificador da Área                                                        63258 non-null  int64  
 5   Dependência Administrativa                                                   63258 non-null  int64  
 6   Localização                           

### Transformação dos dados

* Filtro das escolas de Pelotas

In [7]:
socioeconomicLevel_Pelotas = filterPelotas(socioeconomicLevel)

In [8]:
socioeconomicLevel_Pelotas.head()

,Código da Escola,Nome da Escola,UF,Código do Município,Identificador da Área,Dependência Administrativa,Localização,Quantidade de Alunos com INSE calculado utilizado para o cálculo das médias,Média do Indicador de Nível Socioeconômico dos alunos,Classificação do Indicador de Nível Socioeconômico
0,43000304,COLEGIO TIRADENTES DA BRIGADA MILITAR DE PELOTAS,Rio Grande do Sul,4314407,0,2,1,47,56.08,Grupo 4
1,43100007,COLEGIO GONZAGA,Rio Grande do Sul,4314407,0,4,1,66,62.08,Grupo 5
2,43100031,COLEGIO SAO JOSE,Rio Grande do Sul,4314407,0,4,1,143,62.80,Grupo 6
3,43100058,IFSUL - CAMPUS PELOTAS VISCONDE DA GRACA,Rio Grande do Sul,4314407,0,1,1,56,49.42,Grupo 3
4,43100074,COL ESTADUAL DOM JOAO BRAGA,Rio Grande do Sul,4314407,0,2,1,83,50.14,Grupo 3


In [9]:
socioeconomicLevel_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 10 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   Código da Escola                                                             94 non-null     int64  
 1   Nome da Escola                                                               94 non-null     object 
 2   UF                                                                           94 non-null     object 
 3   Código do Município                                                          94 non-null     int64  
 4   Identificador da Área                                                        94 non-null     int64  
 5   Dependência Administrativa                                                   94 non-null     int64  
 6   Localização                                 

* Colunas desnecessárias

In [10]:
deletingColumns(socioeconomicLevel_Pelotas)

In [15]:
socioeconomicLevel_Pelotas.head()

,Código da Escola,Quantidade de Alunos com INSE calculado utilizado para o cálculo das médias,Média do Indicador de Nível Socioeconômico dos alunos,Classificação do Indicador de Nível Socioeconômico
0,43000304,47,56.08,Grupo 4
1,43100007,66,62.08,Grupo 5
2,43100031,143,62.80,Grupo 6
3,43100058,56,49.42,Grupo 3
4,43100074,83,50.14,Grupo 3


### Teste JSON

In [12]:
rowData = socioeconomicLevel_Pelotas.iloc[ 0 , :  ]

In [13]:
JSONstructure(rowData)

{'Indicador de nível socioeconômico': {'Última atualização': 2015,
  'Quantidade de Alunos com INSE': 47,
  'Média do Indicador de Nível Socioeconômico dos alunos': 56.08,
  'Classificação do Indicador de Nível Socioeconômico': 'Grupo 4'}}

### Carregamento dos dados

* Inserindo no Banco de Dados (MongoDB)

In [16]:
insertIntoDB(socioeconomicLevel_Pelotas, collection)

In [17]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2620d181e7e7953bdb84af'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Média de alunos por turma': {'Última atualização': 2019.0,
  'Educação Infantil': {'Total': nan, 'Creche': nan